In [1]:
import pickle
import pandas as pd
# all_df_mix = pd.read_pickle('all_df_mix.pkl')

In [2]:
df_1 = pd.read_csv('/mnt/d/DATA/data/SAVEE/분노.csv')
df_2 = pd.read_csv('/mnt/d/DATA/data/SAVEE/두려움.csv')
df_3 = pd.read_csv('/mnt/d/DATA/data/SAVEE/슬픔.csv')
df_4 = pd.read_csv('/mnt/d/DATA/data/SAVEE/중립.csv')
df_5 = pd.read_csv('/mnt/d/DATA/data/SAVEE/행복.csv')

df_6 = pd.read_csv('/mnt/d/DATA/data/RAVDESS/01_neutral.csv')
df_7 = pd.read_csv('/mnt/d/DATA/data/RAVDESS/02_calm.csv')
df_8 = pd.read_csv('/mnt/d/DATA/data/RAVDESS/03_happy.csv')
df_9 = pd.read_csv('/mnt/d/DATA/data/RAVDESS/04_sad.csv')
df_10 = pd.read_csv('/mnt/d/DATA/data/RAVDESS/05_angry.csv')

df_11 = pd.read_csv('/mnt/d/DATA/data/EMO-DB/sadness.csv')
df_12 = pd.read_csv('/mnt/d/DATA/data/EMO-DB/anger.csv')
df_13 = pd.read_csv('/mnt/d/DATA/data/EMO-DB/boredom.csv')
df_14 = pd.read_csv('/mnt/d/DATA/data/EMO-DB/happiness.csv')

In [3]:
angry_df = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14])
angry_df

,dir,label
0,/mnt/d/DATA/data/SAVEE/분노/DC_a01.wav,1
1,/mnt/d/DATA/data/SAVEE/분노/DC_a02.wav,1
2,/mnt/d/DATA/data/SAVEE/분노/DC_a03.wav,1
3,/mnt/d/DATA/data/SAVEE/분노/DC_a04.wav,1
4,/mnt/d/DATA/data/SAVEE/분노/DC_a05.wav,1
...,...,...
66,/mnt/d/DATA/data/EMO-DB/happiness/16b02Fd.wav,0
67,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fa.wav,0
68,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fd.wav,0
69,/mnt/d/DATA/data/EMO-DB/happiness/16b09Fb.wav,0


In [4]:
angry_df = angry_df.reset_index()

In [5]:
angry_df

,index,dir,label
0,0,/mnt/d/DATA/data/SAVEE/분노/DC_a01.wav,1
1,1,/mnt/d/DATA/data/SAVEE/분노/DC_a02.wav,1
2,2,/mnt/d/DATA/data/SAVEE/분노/DC_a03.wav,1
3,3,/mnt/d/DATA/data/SAVEE/분노/DC_a04.wav,1
4,4,/mnt/d/DATA/data/SAVEE/분노/DC_a05.wav,1
...,...,...,...
1560,66,/mnt/d/DATA/data/EMO-DB/happiness/16b02Fd.wav,0
1561,67,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fa.wav,0
1562,68,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fd.wav,0
1563,69,/mnt/d/DATA/data/EMO-DB/happiness/16b09Fb.wav,0


In [7]:
import librosa
import numpy as np
import librosa.display
import pyin

def calculate_decibel(audio_path):
    
    audio, sr = librosa.load(audio_path)

    stft = np.abs(librosa.stft(audio))

    power_spec = librosa.power_to_db(stft**2)
    
    max_db = np.max(power_spec)


    return max_db

In [8]:
def calculate_max_pitch(audio_path):
    import librosa
    import scipy.signal as signal
    import numpy as np

    pitch_list = [] 
    audio_sample, sampling_rate = librosa.load(audio_path, sr=16000)

    S = np.abs(librosa.stft(audio_sample, n_fft=1024, hop_length=512, win_length=1024, window=signal.hann))
    pitches, magnitudes = librosa.piptrack(S=S, sr=sampling_rate)

    shape = np.shape(pitches)
    nb_samples = shape[0]
    nb_windows = shape[1]

    for i in range(0, nb_windows):
        index = magnitudes[:, i].argmax()
        pitch = pitches[index, i]
        pitch_list.append(pitch)

    max_pitch = np.max(pitch_list)
    return max_pitch

In [9]:
def calculate_speech_rate(audio_path):
    import librosa

    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path)

    # 소리가 존재하는 구간 검출
    non_silent_intervals = librosa.effects.split(audio, top_db=20)

    # 발화 구간의 길이 계산
    speech_durations = [librosa.get_duration(y=audio[start:end], sr=sr) for start, end in non_silent_intervals]

    # 발화 속도 계산
    average_speech_rate = len(speech_durations) / sum(speech_durations)
    
  
    return average_speech_rate

In [10]:
angry_df['pitch']=''
angry_df['decibel'] =''
angry_df['sps'] =''
angry_df['level']=''

In [11]:
from tqdm import tqdm

In [23]:
for i in tqdm(range(len(angry_df))):
    audio_path = angry_df['dir'][i]
    
    max_db = calculate_decibel(audio_path)
    max_pitch = calculate_max_pitch(audio_path)  
    average_speech_rate = calculate_speech_rate(audio_path)
    
    angry_df['pitch'][i]=max_pitch
    angry_df['decibel'][i]=max_db
    angry_df['sps'][i]=average_speech_rate
    
    if (max_db > 45) or (max_pitch > 22000):
        angry_df['level'][i]=2
    
    elif (max_db > 40) or (max_pitch > 15000):
        angry_df['level'][i]=1

    else:
        angry_df['level'][i]=0

  0%|                                                                                          | 0/1565 [00:00<?, ?it/s]/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'

/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'][i]=average_speech_rate
/tmp/ipykernel_61/2794904285.py:13: SettingWithCopyWarning: 
A value is trying to be set on a c

  1%|▋                                                                                | 14/1565 [00:00<01:32, 16.79it/s]/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'

  1%|█▏                                                                               | 22/1565 [00:01<01:28, 17.41it/s]/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'

/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'][i]=average_speech_rate
/tmp/ipykernel_61/2794904285.py:13: SettingWithCopyWarning: 
A value is trying to be set on a c

...

 98%|█████████████████████████████████████████████████████████████████████████████▌ | 1536/1565 [01:45<00:01, 20.60it/s]/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'

/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'][i]=average_speech_rate
/tmp/ipykernel_61/2794904285.py:16: SettingWithCopyWarning: 
A value is trying to be set on a c

/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'][i]=average_speech_rate
/tmp/ipykernel_61/2794904285.py:16: SettingWithCopyWarning: 
A value is trying to be set on a c

 99%|██████████████████████████████████████████████████████████████████████████████▌| 1557/1565 [01:46<00:00, 21.13it/s]/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'

/tmp/ipykernel_61/2794904285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['pitch'][i]=max_pitch
/tmp/ipykernel_61/2794904285.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['decibel'][i]=max_db
/tmp/ipykernel_61/2794904285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angry_df['sps'][i]=average_speech_rate
/tmp/ipykernel_61/2794904285.py:16: SettingWithCopyWarning: 
A value is trying to be set on a c

In [24]:
angry_df

,index,dir,label,pitch,decibel,sps,level
0,0,/mnt/d/DATA/data/SAVEE/분노/DC_a01.wav,1,3516.135986,48.844872,0.321391,2
1,1,/mnt/d/DATA/data/SAVEE/분노/DC_a02.wav,1,2474.300781,48.106956,0.870028,2
2,2,/mnt/d/DATA/data/SAVEE/분노/DC_a03.wav,1,2680.369873,47.433243,0.916307,2
3,3,/mnt/d/DATA/data/SAVEE/분노/DC_a04.wav,1,2246.690918,52.333885,3.364563,2
4,4,/mnt/d/DATA/data/SAVEE/분노/DC_a05.wav,1,3957.625,51.941433,1.230469,2
...,...,...,...,...,...,...,...
1560,66,/mnt/d/DATA/data/EMO-DB/happiness/16b02Fd.wav,0,3883.151123,40.502289,2.734375,1
1561,67,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fa.wav,0,3889.579102,42.910378,3.28125,1
1562,68,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fd.wav,0,3877.40332,41.722889,1.472356,1
1563,69,/mnt/d/DATA/data/EMO-DB/happiness/16b09Fb.wav,0,3778.615234,41.631248,5.099969,1


In [25]:
angry_df['level'].value_counts()

level
0    992
1    306
2    267
Name: count, dtype: int64

In [26]:
angry_df = angry_df.astype({'level':'int'})
y = angry_df['level']
y_train = np.array( y )

In [27]:
angry_df

,index,dir,label,pitch,decibel,sps,level
0,0,/mnt/d/DATA/data/SAVEE/분노/DC_a01.wav,1,3516.135986,48.844872,0.321391,2
1,1,/mnt/d/DATA/data/SAVEE/분노/DC_a02.wav,1,2474.300781,48.106956,0.870028,2
2,2,/mnt/d/DATA/data/SAVEE/분노/DC_a03.wav,1,2680.369873,47.433243,0.916307,2
3,3,/mnt/d/DATA/data/SAVEE/분노/DC_a04.wav,1,2246.690918,52.333885,3.364563,2
4,4,/mnt/d/DATA/data/SAVEE/분노/DC_a05.wav,1,3957.625,51.941433,1.230469,2
...,...,...,...,...,...,...,...
1560,66,/mnt/d/DATA/data/EMO-DB/happiness/16b02Fd.wav,0,3883.151123,40.502289,2.734375,1
1561,67,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fa.wav,0,3889.579102,42.910378,3.28125,1
1562,68,/mnt/d/DATA/data/EMO-DB/happiness/16b03Fd.wav,0,3877.40332,41.722889,1.472356,1
1563,69,/mnt/d/DATA/data/EMO-DB/happiness/16b09Fb.wav,0,3778.615234,41.631248,5.099969,1


In [28]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [31]:
X = angry_df.drop(['dir', 'label','level'], axis=1)

In [32]:
x_list =[]
for i in range(len(X)):
    p = X['pitch'][i]
    d = X['decibel'][i]
    s = X['sps'][i]
    data =[p,d,s]
    x_list.append(data)
x_list 

[[3516.136, 48.84487, 0.32139109141791045],
 [2474.3008, 48.106956, 0.8700284090909091],
 [2680.3699, 47.433243, 0.9163065159574468],
 [2246.691, 52.333885, 3.36456298828125],
 [3957.625, 51.941433, 1.23046875],
 [3705.9185, 48.493706, 0.9570312499999999],
 [856.41364, 48.028427, 0.67291259765625],
 [3929.6821, 54.42865, 2.8333162006578947],
 [854.431, 45.1545, 2.791341145833333],
 [3926.6462, 47.081146, 2.6300095419847325],
 [609.3919, 49.078594, 0.45333059210526316],
 [2501.3694, 47.308823, 0.44398356958762886],
 [3952.0398, 50.52506, 2.216653262867647],
 [3782.1052, 47.846874, 2.200473312043796],
 [2543.3716, 53.86936, 2.233072916666667],
 [1718.8568, 51.551685, 2.3490767045454546],
 [1984.4487, 50.92472, 1.7698523116438356],
 [1721.1544, 51.679134, 1.025390625],
 [2866.1685, 54.879814, 3.4453124999999996],
 [3937.7097, 54.91143, 3.312800480769231],
 [3936.094, 50.629112, 3.0450994318181817],
 [892.7618, 52.982296, 0.6625600961538461],
 [3958.1184, 55.382767, 2.719983552631579],
 [3

In [33]:
x_list= np.array(x_list)
x_list 

array([[3.51613599e+03, 4.88448715e+01, 3.21391091e-01],
       [2.47430078e+03, 4.81069565e+01, 8.70028409e-01],
       [2.68036987e+03, 4.74332428e+01, 9.16306516e-01],
       ...,
       [3.87740332e+03, 4.17228889e+01, 1.47235577e+00],
       [3.77861523e+03, 4.16312485e+01, 5.09996916e+00],
       [3.90168994e+03, 4.08369827e+01, 2.02665441e+00]])

In [34]:
import xgboost as xgb
from xgboost import XGBClassifier

XGBClassifier= XGBClassifier(
    learning_rate =0.001,
    n_estimators=7000,
    max_depth=1000,
    objective= 'multi:softmax',
    nthread=4,
    seed=27
).fit(x_list, y_train)
XGBClassifier

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1000, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=7000, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='multi:softmax', ...)

In [35]:
import pickle

# Save the model to a file
filename = 'xgb_model_ver5.sav'
pickle.dump(XGBClassifier, open(filename, 'wb'))

In [649]:
audio_path = '/mnt/c/Users/dlrtn/OneDrive/바탕 화면/test/333.wav'

In [36]:
decibel = calculate_decibel(audio_path)
pitch = calculate_pitch(audio_path)
speech_rate = calculate_speech_rate(audio_path)

data = np.array([[pitch,decibel,speech_rate]])




filename = 'xgb_model_ver4.sav'
loaded_model = pickle.load(open(filename, 'rb'))

predicted_label = loaded_model.predict(data)

predicted_probabilities = loaded_model.predict_proba(data)
anger_ratios = predicted_probabilities[0]

print("1st Stage of Anger:", anger_ratios[0])
print("2nd Stage of Anger:", anger_ratios[1])
print("3rd Stage of Anger:", anger_ratios[2])

if predicted_label == 0:
    #1
    print("분노 1단계로 분류됩니다.")
elif predicted_label == 1:
    #2
    print("분노 2단계로 분류됩니다.")
else:
    #3
    print("분노 3단계로 분류됩니다.")

NameError: name 'calculate_pitch' is not defined

In [645]:
speech_rate

1.8197073063380282

In [646]:
pitch

548.6373

In [616]:
decibel

43.069744

In [599]:
c = X.iloc[0:1]

In [594]:
len(X_train)

8087

In [600]:
c

,pitch,decibel,sps
0,357.093323,37.308266,0.916307


In [601]:
audio_f= sc.fit_transform(c)
audio_f

array([[0., 0., 0.]])

In [582]:
audio_path = '/mnt/c/Users/dlrtn/OneDrive/바탕 화면/test/blob.wav'

In [583]:
decibel = calculate_decibel(audio_path)
pitch = calculate_pitch(audio_path)
speech_rate = calculate_speech_rate(audio_path)

In [587]:
data2 = {
'pitch': [pitch],
'decibel': [decibel],
'sps': [speech_rate]
}
df2 = pd.DataFrame(data)
df2

,pitch,decibel,sps
0,475.624115,43.069744,2.533318


In [591]:
df = pd.concat([df2,df])

In [592]:
df

,pitch,decibel,sps
0,475.624115,43.069744,2.533318
0,475.624115,43.069744,2.533318
0,475.624115,43.069744,2.533318


In [593]:
audio_features = sc.fit_transform(df)
audio_features

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
test_emotion_all

In [195]:
import librosa
import librosa.display
import numpy as np
import torch
import pickle
import pandas as pd
import evaluate
import joblib
import pyin
import joblib
import time


from sklearn.metrics import accuracy_score
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)

    global ok_result
    ok_result = [i for i in range(len(predictions)) if predictions[i] == eval_pred.label_ids[i]]
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def preprocess_function(examples):
    audio_arrays = [x for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate
    )
    return inputs

def load_audio(audio_dir):
    sample_rate=16000
    n_fft=2048
    hop_length=512
    n_mels=128
    f_min=0
    f_max=None

    #파일 로드
    waveform, _ = librosa.load(audio_dir, sr=sample_rate)

    # Compute Short-time Fourier transform (STFT)
    stft = torch.stft(torch.Tensor(waveform),
                      n_fft=n_fft,
                      hop_length=hop_length,
                      window=torch.hann_window(n_fft),
                      center=True,
                      normalized=False,
                      onesided=True,
                     return_complex=True)

    # Convert complex STFT to magnitude spectrogram
    mag = torch.abs(stft)

    # Convert magnitude spectrogram to Mel scale
    mel_basis = torch.Tensor(librosa.filters.mel(sr=sample_rate, n_fft=n_fft, n_mels=n_mels, fmin=f_min, fmax=f_max)).to(dtype=torch.float32)

    mel = torch.matmul(mel_basis, mag)

    # Convert to decibels : 로그 스케일(logarithmic scale)
    mel = 20 * torch.log10(torch.clamp(mel, min=1e-5))

    # Normalize between -1 and 1
    mel = torch.clamp((mel + 80) / 80, min=-1, max=1)
    
    mel_flat = torch.flatten(mel)
    mel_array = np.array(mel_flat)
    
    return mel_array

# 테스트SET

In [390]:
test_emotion = pd.read_csv('/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data/test/angry_temp.csv')
test_emotion['dir'][1]

'/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data/test/angry_temp/0011_000373.wav'

In [391]:
test_emotion =test_emotion.reset_index()
test_emotion.drop(columns='index',inplace=True)

In [392]:
def df_make(dataframe):
    dataframe['audio']=''
    for i in range(len(dataframe)):
        audio_dir = dataframe['dir'][i]
        mel_array = load_audio(audio_dir)
        dataframe['audio'][i]=mel_array
    return dataframe
test_emotion = df_make(test_emotion) # audio mel spec
test_emotion['dir'][1]

'/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data/test/angry_temp/0011_000373.wav'

In [393]:
test_emotion['level']=''
test_emotion['pitch']=""
test_emotion['decibel']=""
test_emotion['sps']=""

In [394]:
test_emotion = test_emotion.reset_index()
test_emotion = test_emotion.drop(columns='index')
test_emotion['dir'][1]

'/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data/test/angry_temp/0011_000373.wav'

In [395]:
def calculate_decibel(audio_path):
    # 오디오 파일을 로드합니다.
    audio, sr = librosa.load(audio_path)

    # 시간-주파수 분석을 수행합니다.
    stft = np.abs(librosa.stft(audio))

    # 파워 스펙트럼을 계산합니다.
    power_spec = librosa.power_to_db(stft**2)

    # 평균 데시벨 값을 계산합니다.
    max_db = np.max(power_spec)


    return max_db

def calculate_pitch(audio_path):
    import librosa
    import scipy.signal as signal
    import matplotlib.pyplot as plt
    import numpy as np

    pitch_list=[] 
    audio_sample, sampling_rate = librosa.load(audio_path, sr = None)

    S = np.abs(librosa.stft(audio_sample, n_fft=1024, hop_length=512, win_length = 1024, window=signal.hann))
    pitches, magnitudes = librosa.piptrack(S=S, sr=sampling_rate)

    shape = np.shape(pitches)
    nb_samples = shape[0]
    nb_windows = shape[1]

    for i in range(0, nb_windows):
        index = magnitudes[:,i].argmax()
        pitch = pitches[index,i]
        pitch_list.append(pitch)
        mean_pitch = np.mean(pitch_list)
    return mean_pitch

def calculate_speech_rate(audio_path):
    import librosa

    audio, sr = librosa.load(audio_path)

    non_silent_intervals = librosa.effects.split(audio, top_db=20)

    speech_durations = [librosa.get_duration(y=audio[start:end], sr=sr) for start, end in non_silent_intervals]

    average_speech_rate = len(speech_durations) / sum(speech_durations)
  
    return average_speech_rate

In [396]:
for i in range(len(test_emotion)):
    audio_path = test_emotion['dir'][i]
    
    max_db = calculate_decibel(audio_path)
    mean_pitch = calculate_pitch(audio_path)  
    average_speech_rate = calculate_speech_rate(audio_path)
    
    test_emotion['pitch'][i]=mean_pitch
    test_emotion['decibel'][i]=max_db
    test_emotion['sps'][i]=average_speech_rate
    
    if (max_db > 30) and (mean_pitch > 600) and (average_speech_rate > 3):
        test_emotion['level'][i]=2
    
    elif (max_db > 20) and  (mean_pitch > 500) and (average_speech_rate > 2):
        test_emotion['level'][i]=1
        
    else:
        test_emotion['level'][i]=0
test_emotion

,dir,label,audio,level,pitch,decibel,sps
0,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.4912544, 0.5787911, 0.592181, 0.6265264, 0....",2,861.096741,39.91943,3.588867
1,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[-0.023531055, -0.056408405, -0.13857985, 0.35...",0,551.213013,36.142498,1.266659
2,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[-0.06380663, 0.026813889, 0.44325876, 0.51322...",0,446.575928,39.955647,5.481179
3,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[-0.08897295, -0.08813858, 0.5005465, 0.642963...",0,837.668945,34.997177,1.69999
4,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[-0.06323719, -0.062429525, -0.048125267, -0.0...",1,704.152405,37.243881,2.349077
5,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.48445135, 0.62285423, 0.67544574, 0.6591607...",1,721.711487,39.753262,2.359803
6,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.5176042, 0.6031319, 0.64507216, 0.6019439, ...",0,452.794037,36.13549,2.083858
7,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.52616346, 0.6255754, 0.6480432, 0.62555456,...",1,561.162476,43.140694,4.533306
8,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.6367575, 0.6123093, 0.58269864, 0.5338034, ...",0,389.197418,40.767822,1.745935
9,/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data...,1,"[0.46334243, 0.48729998, 0.4972505, 0.51289904...",1,719.412109,39.987263,2.970097


In [397]:
test_emotion['level'].value_counts()

level
0    24
1    15
2    11
Name: count, dtype: int64

# 머신러닝

In [398]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [399]:
#train
from sklearn.preprocessing import StandardScaler

X = angry_df.drop(['dir', 'label','audio','level'], axis=1)
sc = StandardScaler()
X_train = sc.fit_transform(X)
angry_df = angry_df.astype({'level':'int'})
y = angry_df['level']
y_train = np.array( y )

In [400]:
#test

X2 = test_emotion.drop(['dir', 'label','audio','level'], axis=1)
sc = StandardScaler()
X_test = sc.fit_transform(X2)
test_emotion = test_emotion.astype({'level':'int'})
y2 = test_emotion['level']
y_test = np.array(y2)

In [401]:
#from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()
#y_train = le.fit_transform(y_train)
#y_test = le.fit_transform(y_test)

In [402]:
# 다양한 모델 한번에 테스트
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier



models = []
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('LogisticRegression', LogisticRegression()))
models.append(('XGBClassifier', XGBClassifier()))
models.append(('XGBClassifier', XGBClassifier(
    learning_rate =0.001,
    n_estimators=7000,
    max_depth=1000,
    objective= 'multi:softmax',
    nthread=4,
    seed=27
)))


In [403]:
# 결과를 저장하기 위한 작업
from sklearn.model_selection import KFold, cross_val_score

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=5, random_state=13, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)

    print(name, cv_results.mean(), cv_results.std())

RandomForestClassifier 0.9986241052515906 0.0008699529188722188
DecisionTreeClassifier 0.9991743115789633 0.0011006023643525239
AdaBoostClassifier 0.7550971765927375 0.013623849154638734
GradientBoostingClassifier 0.9991743115789633 0.0011006023643525239
LogisticRegression 0.7361010378892084 0.01394967446568466
XGBClassifier 0.9991743115789633 0.0011006023643525239
XGBClassifier 0.9991743115789633 0.0011006023643525239


In [404]:
from sklearn.metrics import accuracy_score

for name, model in models:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(name, accuracy_score(y_test, pred))

RandomForestClassifier 0.94
DecisionTreeClassifier 0.94
AdaBoostClassifier 0.78
GradientBoostingClassifier 0.94
LogisticRegression 0.84
XGBClassifier 0.94
XGBClassifier 0.94


In [405]:
XGBClassifier= XGBClassifier(
    learning_rate =0.001,
    n_estimators=7000,
    max_depth=1000,
    objective= 'multi:softmax',
    nthread=4,
    seed=27
).fit(X_train, y_train)
XGBClassifier

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1000, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=7000, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='multi:softmax', ...)

In [406]:
pred = XGBClassifier.predict(X_test)

In [407]:
accuracy_score(y_test, pred)

0.94

In [408]:
pred

array([2, 0, 0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1,
       0, 0, 0, 2, 0, 1, 1, 1, 2, 2, 1, 2, 2, 1, 0, 2, 2, 2, 0, 0, 1, 1,
       0, 2, 0, 0, 0, 1], dtype=int32)

In [409]:
y_test

array([2, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1,
       0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 1, 2, 0, 0, 1, 1,
       0, 2, 0, 0, 0, 1])

In [412]:
test_emotion['dir'][38]

'/mnt/c/Users/dlrtn/angry_binary/Main/Data/Data/test/angry_temp/0019_000390.wav'

In [411]:
from xgboost import XGBClassifier
import pickle

# Instantiate the XGBClassifier object
model = XGBClassifier(
    learning_rate=0.001,
    n_estimators=7000,
    max_depth=1000,
    objective='multi:softmax',
    nthread=4,
    seed=27
)

# Train the model
model.fit(X_train, y_train)

# Save the model to a file
filename = 'xgb_model.sav'
pickle.dump(model, open(filename, 'wb'))


파라미터 조정

In [108]:
from sklearn import svm
from joblib import dump, load

dump(Ada, 'Ada.joblib')

['Ada.joblib']

In [198]:
import pickle
pickle.dump(model, open(filename, 'wb'))
# 저장된 모델 파일 경로
filename = 'RandomForestClassifier.sav'

# 모델 불러오기
loaded_model = pickle.load(open(filename, 'rb'))

# 불러온 모델로 예측 수행
pred = loaded_model.predict(X_test)

# 예측 결과 출력
print(pred)


[3 3 1 ... 3 2 2]


# 모델 결정

# 평균치

In [84]:
db_list =[]
for i in range(len(angry_df)):
    audio_path = angry_df['dir'][i]
    
    max_db = calculate_decibel(audio_path)

    db_list.append(max_db)
np.mean(db_list)

1.2410631361792581

In [85]:
pitch_list =[]
for i in range(len(angry_df)):
    audio_path = angry_df['dir'][i]
    
    mean_pitch = calculate_pitch(audio_path) 
    pitch_list.append(mean_pitch)
np.mean(pitch_list)

0.27420310311802637

In [86]:
speech_rate_list =[]
for i in range(len(angry_df)):
    audio_path = angry_df['dir'][i]
    
    average_speech_rate = calculate_speech_rate(audio_path)
    speech_rate_list.append(average_speech_rate)
np.mean(speech_rate_list)

0.2659919663620636